## Dependencies

In [4]:
import torch

import torch.nn as nn
import torch.optim as optim

import torch.nn.functional as F
from torch.autograd import Variable

import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

import torchvision.models as models

import imp
import os
import sys
import math
import utils.training as train_utils; imp.reload(train_utils)
import time
from tqdm import tqdm

In [5]:
DATA_PATH='data/'
RESULTS_PATH='results/'
WEIGHTS_PATH='models/'
PROJECT_NAME='tiramisu'

## Design

**FirstConvLayer**

* 3x3 Conv2D (pad=, stride=, in_chans=3, out_chans=48)

**DenseLayer**

* BatchNorm
* ReLU
* 3x3 Conv2d (pad=, stride=, in_chans=, out_chans=) - "no resolution loss" - padding included
* Dropout (.2)

**DenseBlock**

* Input = FirstConvLayer, TransitionDown, or TransitionUp
* Loop to create L DenseLayers (L=n_layers)
* On TransitionDown we Concat(Input, FinalDenseLayerActivation)
* On TransitionUp we do not Concat with input, instead pass FinalDenseLayerActivation to TransitionUp block

**TransitionDown**

* BatchNorm
* ReLU
* 1x1 Conv2D (pad=, stride=, in_chans=, out_chans=)
* Dropout (0.2)
* 2x2 MaxPooling

**Bottleneck**

* DenseBlock (15 layers)

**TransitionUp**

* 3x3 Transposed Convolution (pad=, stride=2, in_chans=, out_chans=)
* Concat(PreviousDenseBlock, SkipConnection) - from cooresponding DenseBlock on transition down

**FinalBlock**

* 1x1 Conv2d (pad=, stride=, in_chans=256, out_chans=n_classes)
* Softmax

**FCDenseNet103 Architecture**

* input (in_chans=3 for RGB)
* 3x3 ConvLayer (out_chans=48)
* DB (4 layers) + TD
* DB (5 layers) + TD
* DB (7 layers) + TD
* DB (10 layers) + TD
* DB (12 layers) + TD
* Bottleneck (15 layers)
* TU + DB (12 layers)
* TU + DB (10 layers)
* TU + DB (7 layers)
* TU + DB (5 layers)
* TU + DB (4 layers)
* 1x1 ConvLayer (out_chans=n_classes) n_classes=11 for CamVid
* Softmax

## Layers

In [6]:
class DenseLayer(nn.Module):
    def __init__(self, nChannels, growthRate):
        super(DenseLayer, self).__init__()

    def forward(self, x):
        return x
    
class DenseBlock(nn.Module):
    def __init__(self, nChannels, nLayers):
        super(DenseBlock, self).__init__()

    def forward(self, x):
        return x

class TransitionDown(nn.Module):
    def __init__(self, nChannels, nLayers):
        super(TransitionDown, self).__init__()

    def forward(self, x):
        return x
    
class TransitionUp(nn.Module):
    def __init__(self, nChannels, nLayers):
        super(TransitionUp, self).__init__()

    def forward(self, x):
        return x

class Bottleneck(nn.Module):
    def __init__(self, nChannels, growthRate):
        super(Bottleneck, self).__init__()

    def forward(self, x):
        return x

## Model

In [7]:
class FCDenseNet(nn.Module):
    def __init__(self):
        super(FCDenseNet, self).__init__()

    def forward(self, x):
        return x

## Data

In [76]:
BATCH_SIZE=64
CIFAR10_PATH=DATA_PATH+'cifar10/'

In [77]:
torch.cuda.manual_seed(1)

normMean = [0.49139968, 0.48215827, 0.44653124]
normStd = [0.24703233, 0.24348505, 0.26158768]
normTransform = transforms.Normalize(normMean, normStd)

trainTransform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normTransform
])
testTransform = transforms.Compose([
    transforms.ToTensor(),
    normTransform
])

kwargs = {'num_workers': 1, 'pin_memory': True}
trainLoader = DataLoader(
    dset.CIFAR10(root=CIFAR10_PATH, train=True, download=True,
                 transform=trainTransform),
    batch_size=BATCH_SIZE, shuffle=True, **kwargs)
testLoader = DataLoader(
    dset.CIFAR10(root=CIFAR10_PATH, train=False, download=True,
                 transform=testTransform),
    batch_size=BATCH_SIZE, shuffle=False, **kwargs)

Files already downloaded and verified
Files already downloaded and verified


## Train

**Hyperparameters**

* WeightInitialization = HeUniform
* Optimizer = RMSProp
* LR = .001 with exponential decay of 0.995 after each epoch
* Data Augmentation = Random Crops, Vertical Flips
* ValidationSet with early stopping based on IoU or MeanAccuracy with patience of 100 (50 during finetuning)
* WeightDecay = .0001
* Finetune with full-size images, LR = .0001
* Dropout = 0.2
* BatchNorm "we use current batch stats at training, validation, and test time"

**CamVid**

* TrainingSet = 367 frames
* ValidationSet = 101 frames
* TestSet = 233 frames
* Images of resolution 360x480
* Images "Cropped" to 224x224 for training --- center crop?
* FullRes images used for finetuning
* NumberOfClasses = 11 (output)
* BatchSize = 3

**FCDenseNet103**

* GrowthRate = 16 (k, number of filters to each denselayer adds to the ever-growing concatenated output)
* No pretraining

In [82]:
existing_weights_fpath=None
nEpochs=1

net = DenseNet(growthRate=12, depth=40, reduction=1.0, 
                   bottleneck=False, nClasses=10)
net = net.cuda()

optimizer = optim.SGD(net.parameters(), lr=1e-1,
                momentum=0.9, weight_decay=1e-4)

print('  + Number of params: {}'.format(
    sum([p.data.nelement() for p in net.parameters()])))

if existing_weights_fpath:
    startEpoch = train_utils.load_weights(net, existing_weights_fpath)
    endEpoch = startEpoch + nEpochs
    print ('Resume training at epoch: {}'.format(startEpoch))
    if os.path.exists(RESULTS_PATH+'train.csv'): #assume test.csv exists
        append_write = 'a' # append if already exists
    else:
        append_write = 'w' # make a new file if not
    trainF = open(os.path.join(RESULTS_PATH, 'train.csv'), append_write)
    testF = open(os.path.join(RESULTS_PATH, 'test.csv'), append_write)
else:
    print ("Training new model from scratch")
    startEpoch = 1
    endEpoch = nEpochs
    trainF = open(os.path.join(RESULTS_PATH, 'train.csv'), 'w')
    testF = open(os.path.join(RESULTS_PATH, 'test.csv'), 'w')


for epoch in tqdm(range(startEpoch, endEpoch+1)):
    since = time.time()
    train_utils.adjust_opt("sgd", optimizer, epoch)
    train_utils.train(epoch, net, trainLoader, optimizer, trainF)
    train_utils.test(epoch, net, testLoader, optimizer, testF)
    time_elapsed = time.time() - since  
    print('Time {:.0f}m {:.0f}s\n'.format(
        time_elapsed // 60, time_elapsed % 60))
    if epoch != 1:
        os.system('./utils/plot.py {} &'.format(RESULTS_PATH))

trainF.close()
testF.close()


  0%|          | 0/1 [00:00<?, ?it/s]

  + Number of params: 1059298
Training new model from scratch


Epoch 1: Train - Loss: 1.370707	Error: 43.750000


100%|██████████| 1/1 [01:12<00:00, 72.19s/it]

Test - Loss: 1.3404, Error: 4590/10000 (46%)
Time 1m 12s

